# Import reproduction of SR model

In [21]:
%pip install sranodec
import sranodec
# less than period
amp_window_size=24
# (maybe) as same as period
series_window_size=24
# a number enough larger than period
score_window_size=100

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: python-dateutil>=2.7 in c:\users\oxifl\appdata\local\programs\python\python37\lib\site-packages (from matplotlib->sranodec) (2.8.0)



You should consider upgrading via the 'c:\Users\oxifl\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [5]:
import glob
from matplotlib import pyplot
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import pandas as pd
from scipy import stats

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support

import warnings
import os
import datetime
import time
import funcy
import copy
import random
import antropy as ant

from alibi_detect.od import SpectralResidual as AlibiSpectralResidual

In [22]:
warnings.filterwarnings("ignore")

In [23]:
# this is helper method for TS rendering with datapoints
# visualize FP and FN on time series
def ts_confusion_visualization(data_test, pred_val, dataset, filename, modelname):
    x, y, true_val = data_test['timestamp'].tolist(), data_test['value'].tolist(), data_test['is_anomaly'].tolist()
    try:
        x = [datetime.datetime.strptime(x, '%m/%d/%Y %H:%M') for x in x]
    except:
        try:
            x = [datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in x]
        except:
            pass

    fp = [(x[i], y[i]) for i in range(len(true_val)) if true_val[i] == 0 and pred_val[i] == 1]
    fn = [(x[i], y[i]) for i in range(len(true_val)) if true_val[i] == 1 and pred_val[i] == 0]

    fig, ax = plt.subplots()
    ax.plot(x, y, color='grey', lw=0.5, zorder=0)
    ax.scatter([t[0] for t in fp], [t[1] for t in fp], color='r', s=5, zorder=5)
    ax.scatter([t[0] for t in fn], [t[1] for t in fn], color='y', s=5, zorder=5)

    legend_elements = [Line2D([0], [0], color='k', lw=2, label='Correct'),
                       Line2D([0], [0], marker='o', color='r', markersize=5, label='FP'),
                       Line2D([0], [0], marker='o', color='y', markersize=5, label='FN')]

    ax.legend(handles=legend_elements, loc='best')

    pyplot.savefig(f'../results/imgs/{modelname}_{dataset}_{filename}.png')
    pyplot.clf()
    pyplot.close('all')
    plt.close('all')
    del fig
    del ax

In [24]:
# function that finds the indexes of non-anomalies for interpolation 
def interpolation_indexes(mylist, mynumber):
    
    left_neighbour = 0
    right_neighbour = 0
    
    # check left neighbour
    if((mynumber - 1) not in mylist):
        left_neighbour = mynumber - 1
    else:
        min_number = mynumber
        while min_number in mylist:
            min_number = min_number - 1
        left_neighbour = min_number
    
    # check right neighbour
    if((mynumber + 1) not in mylist):
        right_neighbour = mynumber + 1
    else:
        max_number = mynumber
        while max_number in mylist:
            max_number = max_number + 1
        right_neighbour = max_number
    
    return left_neighbour, right_neighbour

In [25]:
def train_anomaly_removal(df_train):
    
    # extract indexes for anomalies
    indexes = list(df_train[df_train.is_anomaly == 1].index)

    # creating a new df that replaces the anomalous samples with interpolation value
    df = pd.DataFrame(columns = df_train.columns)
    for i in range(0, df_train.shape[0]):

        #print(i)

        # add all non-anomalies
        if df_train.is_anomaly[i] == 0:
            df = df.append({'timestamp' : df_train.timestamp[i], 'value' : df_train.value[i], 'is_anomaly' : df_train.is_anomaly[i]},
            ignore_index = True)

        if df_train.is_anomaly[i] == 1:
            if (i+1) < df_train.shape[0] and df_train.is_anomaly[i+1] != 1:
                #print(i)
                value_interpolation = (df_train.value[interpolation_indexes(indexes, i)[0]]+df_train.value[interpolation_indexes(indexes, i)[1]])/2

                df = df.append({'timestamp' : df_train.timestamp[i], 'value': value_interpolation, 'is_anomaly' : 0.0},
            ignore_index = True)
    return df

# Set hyperparameters for train flow

In [26]:
model_name = 'pci'

# decide on windwos with Lorena
anomaly_window = 65
test_window = 65
for_optimization = False

use_drift_adapt = False
drift_detector = None
use_entropy = False
threshold_type = 'static'
if use_entropy:
    threshold_type = 'dynamic'

# TODO discuss averaging
# class averaging type for evaluation metrics calculations
# Calculate metrics globally by counting the total true positives, false negatives and false positives.
avg_type = None

# allowed delay for anomaly shifts during evaluation
evaluation_delay = 7

# Create FFT model

In [27]:
import sys
sys.path.insert(2, '../utils/')
# import importlib
# import os
# import sys

# sys.path.insert(0, os.path.abspath('../module-subdirectory'))

from evaluation import label_evaluation
from fft import detect_anomalies
from dwt_mlead import DWT_MLEAD
from pci import PCIAnomalyDetector

# fft
ifft_parameters: int = 5
context_window_size: int = 21
local_outlier_threshold: float = .6
max_anomaly_window_size: int = 50
max_sign_change_distance: int = 10
random_state: int = 42

# dwt mlead
start_level: int = 3
quantile_epsilon: float = 0.01
random_state: int = 42
use_column_index: int = 0

# from the paper on their timeseries optimal (k,p)
# flactuates around (5, 0.95) and (7, 0.97) so we take approx them
window_size: int = 6
thresholding_p: float = 0.96

# Import SR model

In [ ]:
# import SR as a module from ../utils/sr/
import sys
!{sys.executable} -m pip install ../utils/sr/

In [28]:
from msanomalydetector import THRESHOLD, MAG_WINDOW, SCORE_WINDOW
from msanomalydetector import DetectMode
from msanomalydetector.spectral_residual import SpectralResidual

####################################################################################
# this code is taken from
# https://github.com/microsoft/anomalydetector
# with modifications to account for sliding windows and entropy thresholding
# the modified code is worj from:
# https://github.com/nata1y/tiny-anom-det/tree/main/models/sr
####################################################################################

# initial parameters from the paper
sr_model_params = (THRESHOLD, MAG_WINDOW, SCORE_WINDOW, 99)

# Entropy threshold

In [29]:
# entrop test threshold ##########################################################
def apply_entropy_threshold(svd_entropies, window, boundary_bottom, boundary_up, anomaly_scores, threshold):
    try:
        entropy = ant.svd_entropy(window['value'].tolist(), normalize=True)
    except:
        entropy = (boundary_bottom + boundary_up) / 2

    if entropy < boundary_bottom or entropy > boundary_up:
        extent = stats.percentileofscore(svd_entropies, entropy) / 100.0
        extent = 1.5 - max(extent, 1.0 - extent)
        threshold *= extent
    
    predictions = [1 if a > threshold else 0 for a in anomaly_scores[-window.shape[0]:]]
    return predictions
##################################################################################

# Run experiments

In [31]:
# 'Yahoo_A1Benchmark', 'NAB_realAWSCloudwatch'
# 'sliding_window', 'full_history'

# loop through all models and datasets
for model_name in ['sr', 'fft', 'pci']:
    for dataset in ['Yahoo_A1Benchmark', 'NAB_realAWSCloudwatch']:
        for training_type in ['static', 'sliding_window', 'full_history']:
            print(training_type)
            max_f1 = 0.0
            best_combo = []
            avg_f1 = 0.0
            avg_scs = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
            act = 0
            minl = float('inf')
            maxlen = 1440
            # 'pci',  'dwt_mlead', 'fft', 'sr'
            # set window size equals one week
            # yahoo is hourly, nab and kpi are 5 minute
            if dataset == 'Yahoo_A1Benchmark':
                anomaly_window = 168
            else:
                anomaly_window = 1440

            try:
                stats_full = pd.read_csv(f'../results/scores/{model_name}_{dataset}_{training_type}_stats_{anomaly_window}.csv')
            except:

                test_window = anomaly_window

                if dataset == 'kpi':
                    train_data_path = f'../datasets/kpi/train/'
                else:
                    train_data_path = f'../datasets/{dataset}/'

                for filename in os.listdir(train_data_path):
                    f = os.path.join(train_data_path, filename)
                    data = pd.read_csv(f, engine='python')

                    filename = filename.replace('.csv', '')
                    # print(f'Working with current time series: {filename} in dataset {dataset}')

                    data.rename(columns={'timestamps': 'timestamp', 'anomaly': 'is_anomaly'}, inplace=True)

                    # timestamp preprocessing for kpi -- their are unix timestamps
                    if dataset == 'kpi':
                        data_test = pd.read_csv(os.path.join(f'../datasets/{dataset}/test/', filename + '.csv'))
                        data_test['timestamp'] = data_test['timestamp'].apply(
                            lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
                        data['timestamp'] = data['timestamp'].apply(
                            lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

                        # kpi stores train and test in different ts -- merge them into one to follow structure
                        data = pd.concat([data, data_test], ignore_index=True)

                    # 50-50 train/test split
                    data_train, data_test = np.array_split(data, 2)

                    minl = min(minl, data.shape[0])
                    maxlen = max(maxlen, data.shape[0])
                    act += data['is_anomaly'].tolist().count(1)
                    
                    if data_test['is_anomaly'].tolist().count(1) == 0:
                        continue

                    # set (re)training (== application) windwos based on type
                    if training_type == 'static':
                        data_in_memory_size = anomaly_window
                        pci_window = anomaly_window
                    elif training_type == 'sliding_window':
                        data_in_memory_size = data_train.shape[0]
                        pci_window = data_train.shape[0]
                    elif training_type == 'full_history':
                        data_in_memory_size = 0


                    # train model #######################################################################################
                    start = time.time()
                    fft_threshold = 0

                    if model_name == 'fft':
                        # since no threshold is provided, we are using the max on de-anomalized training set
                        # same tichnique is sued in dynamic models
                        data_train = train_anomaly_removal(data_train)
                        model = detect_anomalies
                        random.seed(random_state)
                        np.random.seed(random_state)
                        anomaly_scores = model(
                                        data_train['value'].to_numpy()
                                    )
                        fft_threshold = max(anomaly_scores)
                    elif model_name == 'pci':
                        model = PCIAnomalyDetector(
                            k=pci_window // 2,
                            p=thresholding_p,
                            calculate_labels=True
                        )
                    elif model_name == 'sr':
                        # THIS IS microsoft model
                        # model = SpectralResidual(series=data_train[['value', 'timestamp']], use_drift=use_drift_adapt,
                        #                 threshold=THRESHOLD, mag_window=MAG_WINDOW,
                        #                 score_window=SCORE_WINDOW, sensitivity=SENSITIVITY,
                        #                 detect_mode=DetectMode.anomaly_only, dataset=dataset,
                        #                 filename=filename, drift_detector=drift_detector,
                        #                 data_in_memory_sz=data_in_memory_size, anomaly_window=anomaly_window)
                        # model.fit()
                        pass

                    end_time = time.time()

                    diff = end_time - start

                    # test model #########################################################################################

                    batch_metrices_f1_entropy = []
                    batch_metrices_f1_no_entropy = []
                    y_pred_total_no_entropy, y_pred_total_entropy = [], []
                    batches_with_anomalies = []
                    idx = 0

                    pred_time = []

                    if for_optimization:
                        data_in_memory = pd.DataFrame([])
                    else:
                        data_in_memory = copy.deepcopy(data_train)

                    for start in range(0, data_test.shape[0], anomaly_window):
                        try:
                            # current window on which TESTING and SCORING is applied
                            window = data_test.iloc[start:start + anomaly_window]
                            # data hold in memory, calculations and predictions are performed across this window
                            data_in_memory = pd.concat([data_in_memory, window])[-data_in_memory_size:]

                            X, y = window['value'], window['is_anomaly']

                            if model_name == 'fft':
                                anomaly_scores = model(
                                        data_in_memory['value'].to_numpy()
                                    )
                                # paper does not provide any way to detect anomaly threshold.
                                # we use same as for lstm
                                y_pred_noe = [1 if x > fft_threshold else 0 for x in anomaly_scores[-window.shape[0]:]]
                                # y_pred_e = apply_entropy_threshold(svd_entropies, window, boundary_bottom, boundary_up, anomaly_scores, fft_threshold)
                            elif model_name == 'sr':
                                # THIS IS MICROSOFT MODEL
                                # model.__series__ = data_in_memory
                                # res = model.predict(data_in_memory, window.shape[0])
                                # y_pred_noe = [1 if x else 0 for x in res['isAnomaly'].tolist()]
                                # # y_pred_e = [1 if x else 0 for x in res['isAnomaly_e'].tolist()]
                                # preds = model.predict(
                                #                 X,
                                #                 t=X['timestamp'],  # array with timesteps, assumes dt=1 between observations if omitted
                                #                 return_instance_score=True
                                #             )

                                # this is reproduction of SR
                                spec = sranodec.Silency(amp_window_size, series_window_size, score_window_size)
                                score = spec.generate_anomaly_score(data_in_memory)
                                index_changes = np.where(score > np.percentile(score, 99))[0]
                                y_pred_noe = [1 if i in index_changes else 0 for i in range(len(score))][-window.shape[0]:]
                                # preds = preds['is_outlier']
                            elif model_name == 'pci':
                                model = PCIAnomalyDetector(
                                    k=data_in_memory.shape[0] // 2,
                                    p=thresholding_p,
                                    calculate_labels=True
                                )
                                anomaly_scores, anomaly_labels = model.detect(data_in_memory['value'].to_numpy())
                                y_pred_noe = anomaly_labels[-window.shape[0]:]
                                # y_pred_e = apply_entropy_threshold(svd_entropies, window, boundary_bottom, boundary_up, anomaly_scores, fft_threshold)

                            idx += 1
                            y_pred_total_no_entropy += [0 if val != 1 else 1 for val in funcy.lflatten(y_pred_noe)][:window.shape[0]]
                            y_pred_noe = y_pred_noe[:window.shape[0]]

                        except Exception as e:
                            raise e

                    # evaluate TS ########################################################################################

                    # calculate batched metrics per *test_window*
                    # for test stats we calculate F1 score for eacj class but use score for anomaly label 
                    # this works because we have binary classification
                    data_reset = data_test.reset_index()['is_anomaly']
                    for i in range(0, len(data_test['is_anomaly']), test_window):
                        # here, met_total will be (precision, recall, f1_score, support)

                        met_total = precision_recall_fscore_support(data_reset[i:i+test_window],
                                                                    y_pred_total_no_entropy[:data_test.shape[0]][i:i+test_window])
                        batch_metrices_f1_no_entropy.append(met_total[2][-1])

                    score = label_evaluation(data_test['is_anomaly'].tolist(), 
                                                    y_pred_total_no_entropy[:data_test.shape[0]], 0)
                    # add entry to stats #######################################################################################

                    try:
                        stats_full = pd.read_csv(f'../results/scores/{model_name}_{dataset}_{training_type}_stats_{anomaly_window}.csv')
                    except:
                        stats_full = pd.DataFrame([])
                        
                    res = {
                        'model': model_name,
                        'ts_name': filename,
                        'precision': met_total[0][-1],
                        'recall': met_total[1][-1],
#                         'window': anomaly_window,
#                         'delay': evaluation_delay,
                        # 'f1_score_entropy': score_entropy,
                        # 'f1_score_entropy_smoothed': smoothed_score_entropy,
#                         'labels_true': data_test['is_anomaly'].tolist(),
#                         'labels_pred': y_pred_total_no_entropy[:data_test.shape[0]]

                    }
                    
                    for delay in range(8):
                        val = label_evaluation(data_test['is_anomaly'].tolist(), 
                                                    y_pred_total_no_entropy[:data_test.shape[0]], delay)
                        avg_scs[delay] += val
                        if delay == 0:
                            avg_f1 += val
                        res[f'f1_score_{delay}'] = val


                    stats_full = stats_full.append(res, ignore_index=True)
                    stats_full.to_csv(f'../results/scores/{model_name}_{dataset}_{training_type}_stats_{anomaly_window}.csv', index=False)

                avg_f1 /= len(os.listdir(train_data_path))
                for i in range(8):
                    avg_scs[i] = avg_scs[i] / len(os.listdir(train_data_path))
                
                # printing average scores
                print(avg_scs)


static


KeyError: -1